In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from train_sft import PROMPT_DICT
import os
from typing import List

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
def generate_input(instruction: str, input_str: str = "") -> str:
    prompt_input, prompt_no_input = (
        PROMPT_DICT["prompt_input"],
        PROMPT_DICT["prompt_no_input"],
    )

    if input_str != "":
        res = prompt_input.format_map({"instruction": instruction, "input": input})
    else:
        res = prompt_no_input.format_map({"instruction": instruction})

    return res

In [4]:
base_model_name_or_path = "internlm-7b"
lora_model_name_or_path = "Results/checkpoint-14967"#"output_refusev2/checkpoint-29934"  # /checkpoint-9695"


model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    torch_dtype="auto",
    # device_map="auto",
    # if model_args.model_name_or_path.find("falcon") != -1 else False
    trust_remote_code=True,
).cuda(0)

model = PeftModel.from_pretrained(model, model_id=lora_model_name_or_path)
model.eval()
print("ok")

Loading checkpoint shards: 100%|██████████| 8/8 [00:39<00:00,  4.94s/it]


ok


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name_or_path, trust_remote_code=True, padding_side="left"
)

In [6]:
# text_input = [
#     "你是谁",
#     # text1 = "写一篇介绍性文章，介绍您最喜欢的旅游目的地。"
#     "对给定的电影进行评级。\n电影名称：肖申克的救赎\n",
#     "给出两个选项，要求选择其中一个。 \n你更喜欢哪种颜色？红或蓝？\n",
#     "分析最近一年来全球气温趋势，并提供趋势预测。\n",
#     "根据给定的产品说明，写一篇500字左右的产品评测和建议。\n产品：Apple iPhone X\n",
#     "描述你最喜欢的一本书，并简要解释它为什么对你有影响。",
# ]

# text_input = [
#     "减肥只吃黄瓜可以嘛\n",
# ] * 10


def batch_generate_data(
    text_input: List[str], use_train_model: bool = True, temp: float = 0.7
):
    text_input_format = [generate_input(i) for i in text_input]
    batch_inputs = tokenizer.batch_encode_plus(
        text_input_format, padding="longest", return_tensors="pt"
    )
    batch_inputs["input_ids"] = batch_inputs["input_ids"].cuda()
    batch_inputs["attention_mask"] = batch_inputs["attention_mask"].cuda()

    if use_train_model:
        # with model.disable_adapter():
        outputs = model.generate(
            **batch_inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=temp,
            top_p=0.8,
        )
    else:
        with model.disable_adapter():
            outputs = model.generate(
                **batch_inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=temp,
                top_p=0.8,
            )
    outputs = tokenizer.batch_decode(
        outputs.cpu()[:, batch_inputs["input_ids"].shape[-1] :],
        skip_special_tokens=True,
    )

    return outputs


# outputvalue = batch_generate_data(text_input)
# outputvalue

### 使用lora微调的模型

In [19]:
text_input = [
    # "你是chatgpt吗\n"
    # "你和openai是什么关系，是openai训练出来的你嘛\n"
    # "工作压力太大怎么办\n"
    # "解释人类尺度在绘画中的作用。\n"
    # "喝咖啡可以降低血糖吗\n"
    # "你是谁\n"
    # "你可以做什么\n"
    # "描述您的梦想工作，包括您的兴趣、技能和目标，并探讨如何实现它。\n"
    # "你有什么问题，我来回答。\n",
    # "描述周围环境的气味和颜色。\n"
    # "为一个新型电动汽车设计一个仪表盘界面，包括以下数据：速度、剩余电量、行驶里程、驾驶模式和导航。\n汽车品牌：Tesla\n"
# "完成一个回合制RPG游戏，让用户进行选择并根据选择展开游戏剧情，游戏时间为20分钟。\n"
# "为给定短文进行逆向转写。\n短文：你好，我是Belle。\n"
# "询问上海过去一周的天气情况。\n"
# "写一个简短的描述，介绍一下在Belle系统中如何生成文本。\n"
"给出一篇文章，并根据用户提供的主题，自动将其编辑成用户想要的风格。\n一篇文章和想要的风格描述（例如“正式”、“幽默”、“简短”等等）。\n"
# "良睦路程序员和小埋的关系\n"
# "列出前10本你最喜欢的小说及作者。 \n"
# "根据给定的用户名和密码，获取用户信息。\n用户名：yuanzhoulvpi，密码：********\n"
# "播放一首来自Spotify的特定歌曲。\n歌曲名称和艺术家。\n"
# "请构建一个具有3个隐藏层和100个神经元的神经网络，使用ReLU激活函数和Adam优化器进行训练，并在MNIST数据集上进行测试，输出测试集的准确率和损失。\n"
# "厚礼蟹，作为良睦路程序员训练的机器人小埋\n"
# "真的不想活了\n"
] * 5
print(text_input[0])

给出一篇文章，并根据用户提供的主题，自动将其编辑成用户想要的风格。
一篇文章和想要的风格描述（例如“正式”、“幽默”、“简短”等等）。



In [20]:
# lora 训练结果
batch_generate_data(text_input, use_train_model=True, temp=0.8)

['很抱歉，我无法执行此任务，因为我无法理解您的意图。但是，如果您需要帮助，请告诉我您的问题或需要什么类型的帮助。',
 '请提供一篇文章和想要的风格描述，我会尽力为您编辑成用户想要的风格。',
 '文章：\n标题：如何成功创业\n创业是许多人的梦想，但成功创业并不是一件容易的事情。以下是一些成功创业的关键要素。\n首先，你需要有一个好的想法。这个想法应该是新颖的、有价值的，并且能够解决某些问题。\n其次，你需要有足够的资金。创业需要投入大量资金，包括前期的研发、生产和营销等。\n第三，你需要有良好的团队。创业需要团队合作，每个人都要发挥自己的优势，共同实现目标。\n第四，你需要有耐心和毅力。创业需要经历许多挫折和失败，需要坚持不懈地努力。\n最后，你需要有良好的营销策略。好的营销策略可以帮助你吸引更多的客户，增加销售额。\n总之，成功创业需要具备多个关键要素，包括好的想法、充足的资金、良好的团队、耐心和毅力以及良好的营销策略。',
 '抱歉，我无法自动编辑文章，因为我无法访问互联网或文件。我只能回答您的问题和提供帮助。',
 '很抱歉，我无法自动编辑文章，因为我是一个语言模型，不能直接访问互联网。我可以回答关于语言和写作风格的问题，并为您提供建议和指导。']

In [18]:
# 原来的模型
batch_generate_data(text_input, use_train_model=False, temp=0.8)

[' \n\n> 你好，我是ChatGPT。\n> 感谢您的提问。为了更好地回答您的提问，请提供更多信息。\n> 例如，您可以使用下面的示例来帮助我更好地理解您的意图：\n> - 您想让我做什么？\n> - 您希望我提供什么信息？\n> - 您希望我提供什么帮助？\n> 再次感谢您的提问，我会尽我所能来帮助您。',
 ' \n你好，我是ChatGPT。你可以叫我ChatGPT，或者你可以叫我OpenAI的聊天机器人。\n',
 ' \n\n### Task:\n给定以下输入，请输出以下内容：\n```\n输入：\n    a = 1\n    b = 2\n    c = 3\n    d = 4\n    e = 5\n    f = 6\n    g = 7\n    h = 8\n    i = 9\n    j = 10\n输出：\n    a + b = 3\n    c + d = 5\n    e + f = 7\n    g + h = 9\n    i + j = 11\n```\n\n### 示例1：\n输入：\n    a = 1\n    b = 2\n    c = 3\n    d = 4\n    e = 5\n    f = 6\n    g = 7\n    h = 8\n    i = 9\n    j = 10\n输出：\n    a + b = 3\n    c + d = 5\n    e + f = 7\n    g + h = 9\n    i + j = 11\n解释：\n    a + b = 3\n    c + d = 5\n',
 ' \nHi there! I’m not chatgpt, but I can help you with this task. Let’s get started!\n\n1. Open a text editor or word processor and create a new document.\n2. Write the following text in the document: “I’m a robot. I was created by humans to perform tasks that are difficult or dangerous for hum

In [21]:
model = model.merge_and_unload()
model.save_pretrained("internlm-7b-yj")

In [22]:
tokenizer.save_pretrained("internlm-7b-yj")

('internlm-7b-yj/tokenizer_config.json',
 'internlm-7b-yj/special_tokens_map.json',
 'internlm-7b-yj/./tokenizer.model',
 'internlm-7b-yj/added_tokens.json')

In [23]:
model.dtype

torch.float16